In [45]:
from pysentimiento import create_analyzer
import os
import pandas as pd

# This notebook was used to explore the sentiment model applied to the initial dataset of our report, where we check if a model that has the sentiment of all justices (using different ratios of probabilities) predicts the sentiment of a judge. This notebook is not used for the report and it's only used for EDA purposes

In [46]:
analyzer = create_analyzer(task="sentiment", lang="en")

In [49]:
# set folder path
folder_path = "data/"

# get list of files in folder
file_list = os.listdir(folder_path)

# create empty list to hold dataframes
df_list = []

# loop through files in folder
for file in file_list:
    # check if file is a CSV
    if file.endswith('.csv'):
        # read CSV file into a pandas dataframe
        df = pd.read_csv(os.path.join(folder_path, file))
        # append dataframe to list
        df_list.append(df)

# concatenate all dataframes in list into a single dataframe
df = pd.concat(df_list, axis=0, ignore_index=True)

C:\Users\aupab\AppData\Local\Temp/ipykernel_16968/1325675326.py:15: DtypeWarning: Columns (60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
C:\Users\aupab\AppData\Local\Temp/ipykernel_16968/1325675326.py:15: DtypeWarning: Columns (60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
C:\Users\aupab\AppData\Local\Temp/ipykernel_16968/1325675326.py:15: DtypeWarning: Columns (58,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
C:\Users\aupab\AppData\Local\Temp/ipykernel_16968/1325675326.py:15: DtypeWarning: Columns (17,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))


In [54]:
df = pd.merge(df, df[['id', 'speaker_type', 'side']], left_on='reply_to', right_on='id', how='left').drop(['id_y'], axis=1)
df.rename(columns={'id_x': 'id', 'speaker_type_x': 'speaker_type', 'speaker_type_y': 'speaker_type_reply_to', 'side_x': 'side', 'side_y': 'side_reply_to'}, inplace=True)
df = pd.merge(df, df[['id', 'speaker_type', 'side']], left_on='addressing', right_on='id', how='left').drop(['id_y'], axis=1)
df.rename(columns={'id_x': 'id', 'speaker_type_x': 'speaker_type', 'speaker_type_y': 'speaker_type_addressing', 'side_x': 'side', 'side_y': 'side_addressing'}, inplace=True)
df['to_same_speaker'] = df['speaker_type_reply_to'] == df['speaker_type_addressing']
#If one is NaN Leave as True - we can still use it. 
df['to_same_speaker'] = df.apply(lambda x: True if pd.isna(x['speaker_type_reply_to']) or pd.isna(x['speaker_type_addressing']) else x['to_same_speaker'], axis=1)
df['to_same_side'] = df['side_reply_to'] == df['side_addressing']
#If one is NaN Leave as True - we can still use it. 
df['to_same_side'] = df.apply(lambda x: True if pd.isna(x['side_reply_to']) or pd.isna(x['side_addressing']) else x['to_same_side'], axis=1)


In [75]:
df[['neg', 'neu', 'pos']] = df['text'].apply(lambda x: pd.Series(analyzer.predict(x).probas))



In [22]:
df_J_to_r = df[(df['speaker_type'] == 'J') & (df['side_addressing'] == 0) & (df['to_same_side'] == True)]
df_J_to_r = df_J_to_r[['case_id', 'pos', 'neu', 'neg', 'win_side']]

C:\Users\aupab\AppData\Local\Temp/ipykernel_16968/69172817.py:1: DtypeWarning: Columns (18,59,61,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('sentiment.csv')


In [8]:
df_J_to_p = df[(df['speaker_type'] == 'J') & (df['side_addressing'] == 1) & (df['to_same_side'] == True)]
df_J_to_p = df_J_to_p[['case_id', 'pos', 'neu', 'neg', 'win_side']]

In [43]:
df_J_to_r = df_J_to_r.groupby(['case_id', 'win_side'])[['pos', 'neu', 'neg']].mean().reset_index()

import numpy as np
ratios = np.arange(0, 50, 0.5)

for i in ratios: 
    df_J_to_r['pos_vs_neg'] = (df_J_to_r['pos']*i > df_J_to_r['neg']).astype(int)
    count = (df_J_to_r['pos_vs_neg'] == df_J_to_r['win_side']).sum()
    print(count/len(df_J_to_r))
    count_1 = df_J_to_r['pos_vs_neg'].sum()
    print(count_1)
    print("*********")


0.3866513233601841
0
*********
0.3866513233601841
4
*********
0.380897583429229
17
*********
0.383199079401611
37
*********
0.4050632911392405
88
*********
0.4131185270425777
153
*********
0.43037974683544306
226
*********
0.44533947065592633
301
*********
0.4579976985040276
362
*********
0.47871116225546606
424
*********
0.4856156501726122
480
*********
0.5086306098964327
528
*********
0.5189873417721519
561
*********
0.5373993095512083
603
*********
0.5489067894131185
631
*********
0.5604142692750288
655
*********
0.5765247410817032
679
*********
0.5799769850402762
702
*********
0.5834292289988493
719
*********
0.5903337169159953
741
*********
0.5926352128883774
745
*********
0.5972382048331415
759
*********
0.5995397008055235
765
*********
0.6041426927502876
771
*********
0.6075949367088608
782
*********
0.6075949367088608
790
*********
0.6087456846950517
799
*********
0.6121979286536249
802
*********
0.6110471806674338
805
*********
0.6087456846950517
809
*********
0.61449942462600

In [42]:
df_J_to_p = df_J_to_p.groupby(['case_id', 'win_side'])[['pos', 'neu', 'neg']].mean().reset_index()

import numpy as np
ratios = np.arange(0, 50, 0.5)

for i in ratios: 
    df_J_to_r['pos_vs_neg'] = (df_J_to_r['pos']*i < df_J_to_r['neg']).astype(int)
    count = (df_J_to_r['pos_vs_neg'] == df_J_to_r['win_side']).sum()
    print(count/len(df_J_to_r))
    count_1 = df_J_to_r['pos_vs_neg'].sum()
    print(count_1)
    print("*********")

0.6133486766398158
869
*********
0.6133486766398158
865
*********
0.619102416570771
852
*********
0.616800920598389
832
*********
0.5949367088607594
781
*********
0.5868814729574223
716
*********
0.569620253164557
643
*********
0.5546605293440736
568
*********
0.5420023014959724
507
*********
0.5212888377445339
445
*********
0.5143843498273878
389
*********
0.49136939010356734
341
*********
0.4810126582278481
308
*********
0.4626006904487917
266
*********
0.45109321058688145
238
*********
0.43958573072497126
214
*********
0.4234752589182969
190
*********
0.42002301495972383
167
*********
0.41657077100115075
150
*********
0.4096662830840046
128
*********
0.40736478711162255
124
*********
0.40276179516685845
110
*********
0.4004602991944764
104
*********
0.3958573072497123
98
*********
0.3924050632911392
87
*********
0.3924050632911392
79
*********
0.3912543153049482
70
*********
0.3878020713463751
67
*********
0.38895281933256615
64
*********
0.3912543153049482
60
*********
0.3855005753

In [69]:
df_J_to_r['ratio_r'] = df_J_to_r['pos'] / df_J_to_r['neg'] 
df_J_to_p['ratio_p'] = df_J_to_p['pos'] / df_J_to_p['neg'] 

df_final = pd.merge(df_J_to_r, df_J_to_p[['case_id', 'ratio_p']], left_on='case_id', right_on='case_id', how='left')


df_final['more_pos'] = (df_final['ratio_p'] > df_final['ratio_r']).astype(int)

count = (df_final['more_pos'] == df_final['win_side']).sum()
print(count/len(df_final))

0.5155350978135789


In [70]:
df_final['win_side'].value_counts()

1.0    533
0.0    336
Name: win_side, dtype: int64